# 고급 LangChain 구성 및 파이프라인 적용하기
* 책 303~306 쪽

<img src='https://raw.githubusercontent.com/corazzon/Mastering-NLP-from-Foundations-to-LLMs/refs/heads/main/cover.png'
     alt="NLP와 LLM 실전 가이드(한빛미디어)"
     style="border: 3px solid gray; box-shadow: 5px 5px 15px rgba(0, 0, 0, 0.3); border-radius: 10px; width: 300px;"   width='300'>


* 저자:  
    - [Lior Gazit](https://www.linkedin.com/in/liorgazit).  
    - [Meysam Ghaffari](https://www.linkedin.com/in/meysam-ghaffari-ph-d-a2553088/).
* 역자:
    - [박조은](https://github.com/corazzon)
* 이 노트북은 다음의 책에서 소개하는 내용입니다.
    - 역서 : NLP와 LLM 실전 가이드(한빛미디어)
    - 원서 : [Mastering NLP from Foundations to LLMs](https://www.amazon.com/dp/1804619183)

colab 실습 :
https://github.com/corazzon/Mastering-NLP-from-Foundations-to-LLMs

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/Mastering-NLP-from-Foundations-to-LLMs/blob/main/Chapter9_notebooks/Ch9_Advanced_LangChain_Configurations_and_Pipeline.ipynb)  


원서 Colab 실습 :
https://github.com/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs   
  
<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs/blob/liors_branch/Chapter9_notebooks/Ch9_Advanced_LangChain_Configurations_and_Pipeline.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**이 노트북의 목적:**  
이 노트북은 8장에서 다룬 파이프라인(**Ch8_Setting_Up_LangChain_Configurations_and_Pipeline.ipynb**)을 개선한 버전입니다.  

우리는 **RAG** 파이프라인을 완성하여, **임베딩**을 생성하고 이를 **벡터 DB**에 저장해 의사의 노트에서 "내부 검색"을 구현합니다.  
이전 노트북과 달리, 여기서는 LLM을 활용해 검색을 수행하며, 잘못된 검색 결과로 인해 발생하는 오류를 방지합니다.  

**필수 사항:**  
* Colab에서 실행 시, 런타임 노트북 설정으로 `Python 3, CPU`를 사용하세요.  
* 이 코드는 OpenAI의 API를 LLM으로 사용하므로, 유료 **API 키**가 필요합니다.  

>*```면책사항: 이 노트북에서 다루는 내용과 아이디어는 저자들 개인의 것이며, 저자들의 고용주의 견해나 지적 재산을 대변하지 않습니다.```*

설치:

In [1]:
# 주의사항:
# 아래 코드에서 Python 패키지 불일치로 인한 오류가 발생하는 경우, 새로운 버전이 원인일 수 있습니다.
# 이런 경우, "default_installations"를 False로 설정하여 원래 이미지로 되돌리세요:
default_installations = True
if default_installations:
    !pip -q install langchain langchain_community
    !pip -q install sentence_transformers
    !pip -q install faiss-cpu
    !pip -q install openai==0.28.1
else:
    import requests
    text_file_path = "requirements__Ch9_Advanced_LangChain_Configurations_and_Pipeline.txt"
    url = "https://raw.githubusercontent.com/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs/main/Chapter9_notebooks/" + text_file_path
    res = requests.get(url)
    with open(text_file_path, "w") as f:
        f.write(res.text)

    !pip install -r requirements__Ch9_Advanced_LangChain_Configurations_and_Pipeline.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7

Imports:

In [2]:
import requests
from langchain.document_loaders import TextLoader
import textwrap
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

코드 설정:

### 모의 의사 노트가 포함된 텍스트 파일 로드  
이 파일들은 우리가 찾으려는 정보를 담고 있습니다.  
이 예제에서는 모든 모의 보고서를 하나의 .CSV 테이블로 결합하여 로딩 과정을 간단하고 짧게 만들었습니다.  

In [3]:
# # 원서
# text_file_path = "mocked_up_physician_records.csv"
# url = "https://raw.githubusercontent.com/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs/main/Chapter8_notebooks/" + text_file_path
# res = requests.get(url)
# with open(text_file_path, "w") as f:
#     f.write(res.text)

### 한국어 버전

In [4]:
text_file_path = "mocked_up_physician_records_ko.csv"
url = "https://raw.githubusercontent.com/corazzon/Mastering-NLP-from-Foundations-to-LLMs/refs/heads/main/Chapter8_notebooks/" + text_file_path
res = requests.get(url)
with open(text_file_path, "w") as f:
    f.write(res.text)

파일의 텍스트 내용을 로드합니다:

In [5]:
# 문서 불러오기
text_loader = TextLoader(text_file_path)
documents = text_loader.load()

LangChain 변수 유형을 확인합니다 (이것은 조작 방법을 이해하는 데 유용합니다):

In [6]:
print(type(documents[0]))

<class 'langchain_core.documents.base.Document'>


문서 개수를 확인합니다:

In [7]:
len(documents)

1

모델에 사용할 텍스트를 확인합니다.

In [8]:
print(documents[0].page_content[0:2000])

"Title: Mocked up record
담당의: ABC 의사
날짜: 2099년 6월 25일
환자 ID: 987654321
주 증상: 복통
현재 질환 이력:
환자 John Anderson 씨(42세, 남성)는 복통을 주 증상으로 내원하였습니다. 기혼자로 아내와 두 자녀와 함께 살고 있습니다. Anderson 씨는 약 2주 전 유럽 출장에서 돌아왔으며, 여행 중 호흡기 증상이나 아픈 사람과의 접촉은 없었다고 합니다.
진료 중 Anderson 씨는 아버지가 60대에 심근경색을 앓은 심혈관 질환 가족력이 있으며, 외할머니는 제2형 당뇨병을 앓았다고 밝혔습니다. 본인은 만성 질환, 수술 경험이나 입원 이력은 없다고 합니다.
주 증상에 대해 Anderson 씨는 우하복부에 둔하고 간헐적인 통증이 있으며, 통증 강도는 10점 만점에 5점 정도라고 설명합니다. 신체 활동 시 통증이 악화되고 식사 후에도 더 심해지는 경향이 있습니다. 구역질, 구토, 배변 습관의 변화와 같은 관련 증상은 없다고 합니다.
제공된 정보를 바탕으로 Anderson 씨의 복통 원인을 파악하기 위한 추가 평가와 진단 검사가 진행될 예정입니다."
"Title: Mocked up record
담당의: ABC 의사
날짜: 2099년 11월 15일
환자 ID: 123456789
주 증상: 피로감과 관절통
현재 질환 이력:
환자 Sarah Thompson 씨(57세, 여성)는 피로감과 관절통을 호소하며 내원하였습니다. 홀로 거주 중인 미망인으로, 최근 해외여행 이력은 없습니다.
진료 중 Thompson 씨는 자매가 류마티스 관절염 진단을 받은 자가면역 질환 가족력이 있다고 밝혔습니다. 또한 본인은 갑상선 기능 저하증이 있어 갑상선 호르몬 대체 요법으로 관리 중이라고 합니다.
주 증상에 대해 Thompson 씨는 일상 활동에 지장을 줄 정도로 지속적이고 심한 피로감을 느끼며, 그 정도는 10점 만점에 8점이라고 표현했습니다. 또한 주로 무릎과 손목에 관절통이 있는데, 아침에 더 심하고 하루 중 활동하면서 점

### 임베딩을 위해 텍스트 데이터 문서별로 나누기

In [9]:
# 우리가 사용하는 데이터 파일에서는 이 짧은 문자열이 서로 다른 임상 보고서들을 구분하는 구분자입니다:
split_text_by = '"Title: Mocked up record'
chunk_size = 700
chunk_overlap = 0

In [10]:
# 텍스트 나누기
text_splitter = CharacterTextSplitter(chunk_size=chunk_size,
                                      chunk_overlap=chunk_overlap,
                                      separator=split_text_by)
splitted_docs = text_splitter.split_documents(documents)

In [11]:
len(splitted_docs)

4

In [12]:
print(splitted_docs[0].page_content)

담당의: ABC 의사
날짜: 2099년 6월 25일
환자 ID: 987654321
주 증상: 복통
현재 질환 이력:
환자 John Anderson 씨(42세, 남성)는 복통을 주 증상으로 내원하였습니다. 기혼자로 아내와 두 자녀와 함께 살고 있습니다. Anderson 씨는 약 2주 전 유럽 출장에서 돌아왔으며, 여행 중 호흡기 증상이나 아픈 사람과의 접촉은 없었다고 합니다.
진료 중 Anderson 씨는 아버지가 60대에 심근경색을 앓은 심혈관 질환 가족력이 있으며, 외할머니는 제2형 당뇨병을 앓았다고 밝혔습니다. 본인은 만성 질환, 수술 경험이나 입원 이력은 없다고 합니다.
주 증상에 대해 Anderson 씨는 우하복부에 둔하고 간헐적인 통증이 있으며, 통증 강도는 10점 만점에 5점 정도라고 설명합니다. 신체 활동 시 통증이 악화되고 식사 후에도 더 심해지는 경향이 있습니다. 구역질, 구토, 배변 습관의 변화와 같은 관련 증상은 없다고 합니다.
제공된 정보를 바탕으로 Anderson 씨의 복통 원인을 파악하기 위한 추가 평가와 진단 검사가 진행될 예정입니다."


### 벡터 데이터베이스에 저장할 임베딩 생성  
허깅페이스의 오픈 소스 모델을 사용합니다.
* https://huggingface.co/sentence-transformers/all-mpnet-base-v2
* MPNet은 BERT, RoBERTa, XLNet 등의 장점을 융합한 최신 Transformer 모델입니다.
* 한국어로는 https://huggingface.co/upskyy/e5-small-korean 를 사용해 봅니다.

In [13]:
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
embeddings = HuggingFaceEmbeddings(model_name="upskyy/e5-small-korean")

<ipython-input-13-4901b9960a1f>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="upskyy/e5-small-korean")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warni

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.77k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/965 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

### 벡터 데이터베이스 생성

<img src="https://python.langchain.com/assets/images/vectorstores-2540b4bc355b966c99b0f02cfdddb273.png">

* 출처 : https://python.langchain.com/docs/concepts/vectorstores/

벡터 데이터베이스로는 FAISS(Facebook AI Similarity Search)를 선택했습니다:

자세한 내용은 다음 링크를 참조하세요:

* https://python.langchain.com/docs/integrations/vectorstores/
* https://python.langchain.com/v0.1/docs/integrations/vectorstores/faiss/#:~:text=Now%2C%20we%20can%20query%20the,similarity_search

In [16]:
vector_db = FAISS.from_documents(splitted_docs, embeddings)

### "내부" 문서를 기반으로 유사도 검색 수행  

**질문 #1: 8월에 출산 예정인 임산부 환자가 있나요?**  

In [17]:
query1 = "8월에 출산 예정인 임산부 환자가 있나요?"
docs = vector_db.similarity_search(query1)
print(textwrap.fill(str(docs[0].page_content), width=100, replace_whitespace=False))

담당의: ABC 의사
날짜: 2099년 7월 10일
환자 ID: 246813579
주 증상: 임신 정기 검진
현재 질환 이력:
환자 Emily Adams 씨(30세, 여성)는 정기
임신 검진을 위해 내원하였습니다. 현재 임신 32주차로, 2099년 8월 27일이 출산 예정일입니다. 기혼자로 남편과 함께 살고 있습니다.
진료 중 Adams 씨는 어머니가 임신
중 임신성 당뇨를 앓았던 가족력이 있다고 밝혔습니다. 본인은 중요한 병력이나 수술 경험, 이전 임신에서의 합병증은 없다고 합니다.
주 증상에 대해 Adams 씨는 임신 3기에 흔히
나타나는 요통, 잦은 소변, 가끔의 속쓰림 등의 불편함을 호소합니다. 질 출혈, 심한 복통, 태아 움직임의 특이한 변화는 없다고 합니다. 임신 중 전반적인 건강 관리를 위해 균형
잡힌 식단과 규칙적인 운동을 유지하고 있다고 합니다.
제공된 정보를 바탕으로 Adams 씨의 임신 경과를 확인하기 위한 정기 산전 검사가 진행될 예정입니다. 혈압, 체중 증가,
자궁저 높이 측정, 태아 심장 박동 모니터링 등을 평가할 것입니다. 건강한 출산과 순조로운 육아를 위해 출산 준비, 모유 수유, 산후 관리에 대한 상담도 함께 이루어질
예정입니다."


**[오류의 예!] 질문 #2: 출산 예정일이 9월인 임산부가 있나요?**  
이것은 유사성 검색이 **틀린 결과**를 제공하는 예입니다.  
질문과 유사한 텍스트를 제공하긴 했지만, 이 사례는 유사성이 정답을 맞히는 것과는 다르다는 점을 보여줍니다.  

In [18]:
query2 = "출산 예정일이 9월인 임산부가 있나요?"
docs = vector_db.similarity_search(query2)
print(textwrap.fill(str(docs[0].page_content), width=100, replace_whitespace=False))

담당의: ABC 의사
날짜: 2099년 7월 10일
환자 ID: 246813579
주 증상: 임신 정기 검진
현재 질환 이력:
환자 Emily Adams 씨(30세, 여성)는 정기
임신 검진을 위해 내원하였습니다. 현재 임신 32주차로, 2099년 8월 27일이 출산 예정일입니다. 기혼자로 남편과 함께 살고 있습니다.
진료 중 Adams 씨는 어머니가 임신
중 임신성 당뇨를 앓았던 가족력이 있다고 밝혔습니다. 본인은 중요한 병력이나 수술 경험, 이전 임신에서의 합병증은 없다고 합니다.
주 증상에 대해 Adams 씨는 임신 3기에 흔히
나타나는 요통, 잦은 소변, 가끔의 속쓰림 등의 불편함을 호소합니다. 질 출혈, 심한 복통, 태아 움직임의 특이한 변화는 없다고 합니다. 임신 중 전반적인 건강 관리를 위해 균형
잡힌 식단과 규칙적인 운동을 유지하고 있다고 합니다.
제공된 정보를 바탕으로 Adams 씨의 임신 경과를 확인하기 위한 정기 산전 검사가 진행될 예정입니다. 혈압, 체중 증가,
자궁저 높이 측정, 태아 심장 박동 모니터링 등을 평가할 것입니다. 건강한 출산과 순조로운 육아를 위해 출산 준비, 모유 수유, 산후 관리에 대한 상담도 함께 이루어질
예정입니다."


**질문 #3: 최근에 여행을 다녀온 환자는 누구인가요?**

In [19]:
query3 = "최근에 여행을 다녀온 환자는 누구인가요?"
docs = vector_db.similarity_search(query3)
print(textwrap.fill(str(docs[0].page_content), width=100, replace_whitespace=False))

담당의: ABC 의사
날짜: 2099년 11월 15일
환자 ID: 123456789
주 증상: 피로감과 관절통
현재 질환 이력:
환자 Sarah Thompson 씨(57세,
여성)는 피로감과 관절통을 호소하며 내원하였습니다. 홀로 거주 중인 미망인으로, 최근 해외여행 이력은 없습니다.
진료 중 Thompson 씨는 자매가 류마티스 관절염 진단을 받은
자가면역 질환 가족력이 있다고 밝혔습니다. 또한 본인은 갑상선 기능 저하증이 있어 갑상선 호르몬 대체 요법으로 관리 중이라고 합니다.
주 증상에 대해 Thompson 씨는 일상
활동에 지장을 줄 정도로 지속적이고 심한 피로감을 느끼며, 그 정도는 10점 만점에 8점이라고 표현했습니다. 또한 주로 무릎과 손목에 관절통이 있는데, 아침에 더 심하고 하루 중
활동하면서 점차 나아진다고 합니다. 관절 부위의 붓기나 붉어짐은 없다고 합니다.
임상 소견을 바탕으로 Thompson 씨의 피로감과 관절통의 원인을 찾기 위한 추가 검사가
필요합니다. 혈액 검사, 영상 검사 및 필요시 전문의 상담을 진행할 예정입니다."


**질문 #4: 검사실 검사가 필요한 환자는 누구인가요?**

In [20]:
query4 = "검사실 검사가 필요한 환자는 누구인가요?"
docs = vector_db.similarity_search(query4)
print(textwrap.fill(str(docs[0].page_content), width=100, replace_whitespace=False))

담당의: ABC 의사
날짜: 2099년 11월 15일
환자 ID: 123456789
주 증상: 피로감과 관절통
현재 질환 이력:
환자 Sarah Thompson 씨(57세,
여성)는 피로감과 관절통을 호소하며 내원하였습니다. 홀로 거주 중인 미망인으로, 최근 해외여행 이력은 없습니다.
진료 중 Thompson 씨는 자매가 류마티스 관절염 진단을 받은
자가면역 질환 가족력이 있다고 밝혔습니다. 또한 본인은 갑상선 기능 저하증이 있어 갑상선 호르몬 대체 요법으로 관리 중이라고 합니다.
주 증상에 대해 Thompson 씨는 일상
활동에 지장을 줄 정도로 지속적이고 심한 피로감을 느끼며, 그 정도는 10점 만점에 8점이라고 표현했습니다. 또한 주로 무릎과 손목에 관절통이 있는데, 아침에 더 심하고 하루 중
활동하면서 점차 나아진다고 합니다. 관절 부위의 붓기나 붉어짐은 없다고 합니다.
임상 소견을 바탕으로 Thompson 씨의 피로감과 관절통의 원인을 찾기 위한 추가 검사가
필요합니다. 혈액 검사, 영상 검사 및 필요시 전문의 상담을 진행할 예정입니다."


****
다음 단계 전에 메모리를 정리합니다 (로컬 호스팅된 LLM을 선택한 경우에 유용):

In [21]:
import sys

local_vars = list(locals().items())
for var, obj in local_vars:
  if(sys.getsizeof(obj)) > 999:
    print(var, sys.getsizeof(obj))

_i1 1650
TextLoader 1688
CharacterTextSplitter 1688
HuggingFaceEmbeddings 1688
FAISS 1688


In [22]:
import gc
del CharacterTextSplitter
del HuggingFaceEmbeddings
del TextLoader
del FAISS
gc.collect()

349

****

# 8장 노트북의 개선: 요청 처리를 위한 LLM 설정  
이제 해당 파이프라인을 개선하겠습니다. 단순히 유사성 검색 결과를 의사에게 제공하는 것에 그치지 않고, 요청과 유사한 내용으로 간주된 결과를 바탕으로 LLM을 활용하여 이를 검토하고, 실제로 의사에게 적합한 결과를 판별해낼 것입니다.  

OpenAI API key:  
**문자열 형태로 아래 "..."에 OpenAI에서 발급받은 key를 입력해 주세요!**  


Colab 보안 비밀 설정은 왼쪽 열쇠 모양의 아이콘을 클릭하면 나옵니다.
<img src="https://i.imgur.com/7P383n4.png" width="500">




In [23]:
openai_api_key = "..."
# colab 보안 비밀 설정 사용시 주석 해제 후 사용
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

In [24]:
!pip -q install openai gpt4all==1.0.12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 3.8 MB/s eta 0:00:00


In [ ]:
# !wget https://huggingface.co/TheBloke/Nous-Hermes-13B-GGML/resolve/main/nous-hermes-13b.ggmlv3.q4_0.bin

In [27]:
import os
import langchain
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub
from langchain.chat_models import ChatOpenAI
from langchain.llms import GPT4All

### LLM 설정: 유료 LLM(OpenAI의 GPT)과 무료 LLM(Hugging Face 모델) 중 선택  

In [28]:
paid_vs_free = "paid"

# GPT4all .bin 파일의 경로 (Google Colab에서 실행하기에 적합):
path_to_bin = "./nous-hermes-13b.ggmlv3.q4_0.bin"

# 백엔드 LLM:
# "gptj", "llama" 등 다양한 모델 중 선택 합니다.
backend_llm = "llama"

In [29]:
if paid_vs_free == "paid":
    os.environ["OPENAI_API_KEY"] = openai_api_key
    llm = ChatOpenAI()
elif paid_vs_free == "free":
    llm = GPT4All(
        model=path_to_bin,
        max_tokens=1000,
        # backend=backend_llm,
        verbose=False)

<ipython-input-29-5e90faaaf365>:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI()


### QA 체인 생성  
`load_qa_chain()`을 통해 RAG 프레임워크를 구성합니다. 이 기능은 다양한 텍스트 문서를 입력받아 검색 가능한 형태로 준비하며, 사용자 프롬프트를 분석하여 관련된 텍스트를 찾아냅니다. 찾아낸 텍스트는 선택된 LLM에 제공되어, LLM이 적절한 맥락을 바탕으로 프롬프트에 답변할 수 있게 합니다.

In [30]:
chain = load_qa_chain(llm, chain_type="stuff")

<ipython-input-30-1b5424917bc0>:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff")


### 동일한 요구 사항을 기반으로 검색하되, 임베딩 유사성 대신 LLM을 "두뇌"로 활용하여 검색

**Question #1: 8월에 출산 예정인 임산부 환자가 있나요?**  

In [31]:
import langchain
langchain.debug = True

In [36]:
current_query = query1
print(current_query)
docs = vector_db.similarity_search(current_query, k=4)
response = chain.invoke({"input_documents": docs, "question": current_query})
response

8월에 출산 예정인 임산부 환자가 있나요?
[chain/start] [chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "8월에 출산 예정인 임산부 환자가 있나요?",
  "context": "담당의: ABC 의사\n날짜: 2099년 7월 10일\n환자 ID: 246813579\n주 증상: 임신 정기 검진\n현재 질환 이력:\n환자 Emily Adams 씨(30세, 여성)는 정기 임신 검진을 위해 내원하였습니다. 현재 임신 32주차로, 2099년 8월 27일이 출산 예정일입니다. 기혼자로 남편과 함께 살고 있습니다.\n진료 중 Adams 씨는 어머니가 임신 중 임신성 당뇨를 앓았던 가족력이 있다고 밝혔습니다. 본인은 중요한 병력이나 수술 경험, 이전 임신에서의 합병증은 없다고 합니다.\n주 증상에 대해 Adams 씨는 임신 3기에 흔히 나타나는 요통, 잦은 소변, 가끔의 속쓰림 등의 불편함을 호소합니다. 질 출혈, 심한 복통, 태아 움직임의 특이한 변화는 없다고 합니다. 임신 중 전반적인 건강 관리를 위해 균형 잡힌 식단과 규칙적인 운동을 유지하고 있다고 합니다.\n제공된 정보를 바탕으로 Adams 씨의 임신 경과를 확인하기 위한 정기 산전 검사가 진행될 예정입니다. 혈압, 체중 증가, 자궁저 높이 측정, 태아 심장 박동 모니터링 등을 평가할 것입니다. 건강한 출산과 순조로운 육아를 위해 출산 준비, 모유 수유, 산후 관리에 대한 상담도 함께 이루어질 예정입니다.\"\n\n담당의: ABC 의사\n날짜: 2099년 11월 15일\n환자 ID: 123456789\n주 증상: 피로감과 관절통\n현재 질환 이력:\n환자 Sarah Thompson 씨(57세, 여성)는 피로감과 관절통을 호소하며 내원하였습니다. 홀

{'input_documents': [Document(id='b29ba404-e5c2-4115-befe-7790fc8eb705', metadata={'source': 'mocked_up_physician_records_ko.csv'}, page_content='담당의: ABC 의사\n날짜: 2099년 7월 10일\n환자 ID: 246813579\n주 증상: 임신 정기 검진\n현재 질환 이력:\n환자 Emily Adams 씨(30세, 여성)는 정기 임신 검진을 위해 내원하였습니다. 현재 임신 32주차로, 2099년 8월 27일이 출산 예정일입니다. 기혼자로 남편과 함께 살고 있습니다.\n진료 중 Adams 씨는 어머니가 임신 중 임신성 당뇨를 앓았던 가족력이 있다고 밝혔습니다. 본인은 중요한 병력이나 수술 경험, 이전 임신에서의 합병증은 없다고 합니다.\n주 증상에 대해 Adams 씨는 임신 3기에 흔히 나타나는 요통, 잦은 소변, 가끔의 속쓰림 등의 불편함을 호소합니다. 질 출혈, 심한 복통, 태아 움직임의 특이한 변화는 없다고 합니다. 임신 중 전반적인 건강 관리를 위해 균형 잡힌 식단과 규칙적인 운동을 유지하고 있다고 합니다.\n제공된 정보를 바탕으로 Adams 씨의 임신 경과를 확인하기 위한 정기 산전 검사가 진행될 예정입니다. 혈압, 체중 증가, 자궁저 높이 측정, 태아 심장 박동 모니터링 등을 평가할 것입니다. 건강한 출산과 순조로운 육아를 위해 출산 준비, 모유 수유, 산후 관리에 대한 상담도 함께 이루어질 예정입니다."'),
  Document(id='06189ba5-40aa-4bf6-81f6-d6344f261d65', metadata={'source': 'mocked_up_physician_records_ko.csv'}, page_content='담당의: ABC 의사\n날짜: 2099년 11월 15일\n환자 ID: 123456789\n주 증상: 피로감과 관절통\n현재 질환 이력:\n환자 Sarah Thompson 씨(57세, 여성)는 피로감과 관

In [37]:
docs[0]

Document(id='b29ba404-e5c2-4115-befe-7790fc8eb705', metadata={'source': 'mocked_up_physician_records_ko.csv'}, page_content='담당의: ABC 의사\n날짜: 2099년 7월 10일\n환자 ID: 246813579\n주 증상: 임신 정기 검진\n현재 질환 이력:\n환자 Emily Adams 씨(30세, 여성)는 정기 임신 검진을 위해 내원하였습니다. 현재 임신 32주차로, 2099년 8월 27일이 출산 예정일입니다. 기혼자로 남편과 함께 살고 있습니다.\n진료 중 Adams 씨는 어머니가 임신 중 임신성 당뇨를 앓았던 가족력이 있다고 밝혔습니다. 본인은 중요한 병력이나 수술 경험, 이전 임신에서의 합병증은 없다고 합니다.\n주 증상에 대해 Adams 씨는 임신 3기에 흔히 나타나는 요통, 잦은 소변, 가끔의 속쓰림 등의 불편함을 호소합니다. 질 출혈, 심한 복통, 태아 움직임의 특이한 변화는 없다고 합니다. 임신 중 전반적인 건강 관리를 위해 균형 잡힌 식단과 규칙적인 운동을 유지하고 있다고 합니다.\n제공된 정보를 바탕으로 Adams 씨의 임신 경과를 확인하기 위한 정기 산전 검사가 진행될 예정입니다. 혈압, 체중 증가, 자궁저 높이 측정, 태아 심장 박동 모니터링 등을 평가할 것입니다. 건강한 출산과 순조로운 육아를 위해 출산 준비, 모유 수유, 산후 관리에 대한 상담도 함께 이루어질 예정입니다."')

**[OpenAI의 LLM이 오류를 감지하고 피했습니다!] 질문 #2: 9월에 출산 예정인 임산부가 있나요?**  
다만, 양자화되어 "성능이 저하된" 일부 무료 LLM들은 실패할 수 있으며, 9월 출산 예정일에 대해 물었음에도 8월 출산 예정일을 답변으로 제시할 수 있습니다.

In [40]:
current_query = query2
print(current_query)
docs = vector_db.similarity_search(current_query, k=3)
response = chain.invoke({"input_documents": docs, "question": current_query})
response

출산 예정일이 9월인 임산부가 있나요?
[chain/start] [chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "출산 예정일이 9월인 임산부가 있나요?",
  "context": "담당의: ABC 의사\n날짜: 2099년 7월 10일\n환자 ID: 246813579\n주 증상: 임신 정기 검진\n현재 질환 이력:\n환자 Emily Adams 씨(30세, 여성)는 정기 임신 검진을 위해 내원하였습니다. 현재 임신 32주차로, 2099년 8월 27일이 출산 예정일입니다. 기혼자로 남편과 함께 살고 있습니다.\n진료 중 Adams 씨는 어머니가 임신 중 임신성 당뇨를 앓았던 가족력이 있다고 밝혔습니다. 본인은 중요한 병력이나 수술 경험, 이전 임신에서의 합병증은 없다고 합니다.\n주 증상에 대해 Adams 씨는 임신 3기에 흔히 나타나는 요통, 잦은 소변, 가끔의 속쓰림 등의 불편함을 호소합니다. 질 출혈, 심한 복통, 태아 움직임의 특이한 변화는 없다고 합니다. 임신 중 전반적인 건강 관리를 위해 균형 잡힌 식단과 규칙적인 운동을 유지하고 있다고 합니다.\n제공된 정보를 바탕으로 Adams 씨의 임신 경과를 확인하기 위한 정기 산전 검사가 진행될 예정입니다. 혈압, 체중 증가, 자궁저 높이 측정, 태아 심장 박동 모니터링 등을 평가할 것입니다. 건강한 출산과 순조로운 육아를 위해 출산 준비, 모유 수유, 산후 관리에 대한 상담도 함께 이루어질 예정입니다.\"\n\n담당의: ABC 의사\n날짜: 2099년 6월 25일\n환자 ID: 987654321\n주 증상: 복통\n현재 질환 이력:\n환자 John Anderson 씨(42세, 남성)는 복통을 주 증상으로 내원하였습니다. 기혼자로 아내와 두 자녀와 함께

{'input_documents': [Document(id='b29ba404-e5c2-4115-befe-7790fc8eb705', metadata={'source': 'mocked_up_physician_records_ko.csv'}, page_content='담당의: ABC 의사\n날짜: 2099년 7월 10일\n환자 ID: 246813579\n주 증상: 임신 정기 검진\n현재 질환 이력:\n환자 Emily Adams 씨(30세, 여성)는 정기 임신 검진을 위해 내원하였습니다. 현재 임신 32주차로, 2099년 8월 27일이 출산 예정일입니다. 기혼자로 남편과 함께 살고 있습니다.\n진료 중 Adams 씨는 어머니가 임신 중 임신성 당뇨를 앓았던 가족력이 있다고 밝혔습니다. 본인은 중요한 병력이나 수술 경험, 이전 임신에서의 합병증은 없다고 합니다.\n주 증상에 대해 Adams 씨는 임신 3기에 흔히 나타나는 요통, 잦은 소변, 가끔의 속쓰림 등의 불편함을 호소합니다. 질 출혈, 심한 복통, 태아 움직임의 특이한 변화는 없다고 합니다. 임신 중 전반적인 건강 관리를 위해 균형 잡힌 식단과 규칙적인 운동을 유지하고 있다고 합니다.\n제공된 정보를 바탕으로 Adams 씨의 임신 경과를 확인하기 위한 정기 산전 검사가 진행될 예정입니다. 혈압, 체중 증가, 자궁저 높이 측정, 태아 심장 박동 모니터링 등을 평가할 것입니다. 건강한 출산과 순조로운 육아를 위해 출산 준비, 모유 수유, 산후 관리에 대한 상담도 함께 이루어질 예정입니다."'),
  Document(id='593825c1-cb24-4734-a14d-741cba3b4c7e', metadata={'source': 'mocked_up_physician_records_ko.csv'}, page_content='담당의: ABC 의사\n날짜: 2099년 6월 25일\n환자 ID: 987654321\n주 증상: 복통\n현재 질환 이력:\n환자 John Anderson 씨(42세, 남성)는 복통을 주 증상으로 내원하

**질문 #3: 최근에 여행을 다녀온 환자는 누구인가요?**

In [41]:
current_query = query3
print(current_query)
docs = vector_db.similarity_search(current_query)
response = chain.invoke({"input_documents": docs, "question": current_query})
response

최근에 여행을 다녀온 환자는 누구인가요?
[chain/start] [chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "최근에 여행을 다녀온 환자는 누구인가요?",
  "context": "담당의: ABC 의사\n날짜: 2099년 11월 15일\n환자 ID: 123456789\n주 증상: 피로감과 관절통\n현재 질환 이력:\n환자 Sarah Thompson 씨(57세, 여성)는 피로감과 관절통을 호소하며 내원하였습니다. 홀로 거주 중인 미망인으로, 최근 해외여행 이력은 없습니다.\n진료 중 Thompson 씨는 자매가 류마티스 관절염 진단을 받은 자가면역 질환 가족력이 있다고 밝혔습니다. 또한 본인은 갑상선 기능 저하증이 있어 갑상선 호르몬 대체 요법으로 관리 중이라고 합니다.\n주 증상에 대해 Thompson 씨는 일상 활동에 지장을 줄 정도로 지속적이고 심한 피로감을 느끼며, 그 정도는 10점 만점에 8점이라고 표현했습니다. 또한 주로 무릎과 손목에 관절통이 있는데, 아침에 더 심하고 하루 중 활동하면서 점차 나아진다고 합니다. 관절 부위의 붓기나 붉어짐은 없다고 합니다.\n임상 소견을 바탕으로 Thompson 씨의 피로감과 관절통의 원인을 찾기 위한 추가 검사가 필요합니다. 혈액 검사, 영상 검사 및 필요시 전문의 상담을 진행할 예정입니다.\"\n\n담당의: ABC 의사\n날짜: 2099년 6월 25일\n환자 ID: 987654321\n주 증상: 복통\n현재 질환 이력:\n환자 John Anderson 씨(42세, 남성)는 복통을 주 증상으로 내원하였습니다. 기혼자로 아내와 두 자녀와 함께 살고 있습니다. Anderson 씨는 약 2주 전 유럽 출장에서 돌아왔으며, 여행 중 호흡기 증상이나 아픈 사람

{'input_documents': [Document(id='06189ba5-40aa-4bf6-81f6-d6344f261d65', metadata={'source': 'mocked_up_physician_records_ko.csv'}, page_content='담당의: ABC 의사\n날짜: 2099년 11월 15일\n환자 ID: 123456789\n주 증상: 피로감과 관절통\n현재 질환 이력:\n환자 Sarah Thompson 씨(57세, 여성)는 피로감과 관절통을 호소하며 내원하였습니다. 홀로 거주 중인 미망인으로, 최근 해외여행 이력은 없습니다.\n진료 중 Thompson 씨는 자매가 류마티스 관절염 진단을 받은 자가면역 질환 가족력이 있다고 밝혔습니다. 또한 본인은 갑상선 기능 저하증이 있어 갑상선 호르몬 대체 요법으로 관리 중이라고 합니다.\n주 증상에 대해 Thompson 씨는 일상 활동에 지장을 줄 정도로 지속적이고 심한 피로감을 느끼며, 그 정도는 10점 만점에 8점이라고 표현했습니다. 또한 주로 무릎과 손목에 관절통이 있는데, 아침에 더 심하고 하루 중 활동하면서 점차 나아진다고 합니다. 관절 부위의 붓기나 붉어짐은 없다고 합니다.\n임상 소견을 바탕으로 Thompson 씨의 피로감과 관절통의 원인을 찾기 위한 추가 검사가 필요합니다. 혈액 검사, 영상 검사 및 필요시 전문의 상담을 진행할 예정입니다."'),
  Document(id='593825c1-cb24-4734-a14d-741cba3b4c7e', metadata={'source': 'mocked_up_physician_records_ko.csv'}, page_content='담당의: ABC 의사\n날짜: 2099년 6월 25일\n환자 ID: 987654321\n주 증상: 복통\n현재 질환 이력:\n환자 John Anderson 씨(42세, 남성)는 복통을 주 증상으로 내원하였습니다. 기혼자로 아내와 두 자녀와 함께 살고 있습니다. Anderson 씨는 약 2주 전 유럽 출장에서 돌아왔으며

In [42]:
docs[1]

Document(id='593825c1-cb24-4734-a14d-741cba3b4c7e', metadata={'source': 'mocked_up_physician_records_ko.csv'}, page_content='담당의: ABC 의사\n날짜: 2099년 6월 25일\n환자 ID: 987654321\n주 증상: 복통\n현재 질환 이력:\n환자 John Anderson 씨(42세, 남성)는 복통을 주 증상으로 내원하였습니다. 기혼자로 아내와 두 자녀와 함께 살고 있습니다. Anderson 씨는 약 2주 전 유럽 출장에서 돌아왔으며, 여행 중 호흡기 증상이나 아픈 사람과의 접촉은 없었다고 합니다.\n진료 중 Anderson 씨는 아버지가 60대에 심근경색을 앓은 심혈관 질환 가족력이 있으며, 외할머니는 제2형 당뇨병을 앓았다고 밝혔습니다. 본인은 만성 질환, 수술 경험이나 입원 이력은 없다고 합니다.\n주 증상에 대해 Anderson 씨는 우하복부에 둔하고 간헐적인 통증이 있으며, 통증 강도는 10점 만점에 5점 정도라고 설명합니다. 신체 활동 시 통증이 악화되고 식사 후에도 더 심해지는 경향이 있습니다. 구역질, 구토, 배변 습관의 변화와 같은 관련 증상은 없다고 합니다.\n제공된 정보를 바탕으로 Anderson 씨의 복통 원인을 파악하기 위한 추가 평가와 진단 검사가 진행될 예정입니다."')

**질문 #4: 검사실 검사가 필요한 환자는 누구인가요?**

In [43]:
current_query = query4
print(current_query)
docs = vector_db.similarity_search(current_query)
response = chain.invoke({"input_documents": docs, "question": current_query})
response

검사실 검사가 필요한 환자는 누구인가요?
[chain/start] [chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "검사실 검사가 필요한 환자는 누구인가요?",
  "context": "담당의: ABC 의사\n날짜: 2099년 11월 15일\n환자 ID: 123456789\n주 증상: 피로감과 관절통\n현재 질환 이력:\n환자 Sarah Thompson 씨(57세, 여성)는 피로감과 관절통을 호소하며 내원하였습니다. 홀로 거주 중인 미망인으로, 최근 해외여행 이력은 없습니다.\n진료 중 Thompson 씨는 자매가 류마티스 관절염 진단을 받은 자가면역 질환 가족력이 있다고 밝혔습니다. 또한 본인은 갑상선 기능 저하증이 있어 갑상선 호르몬 대체 요법으로 관리 중이라고 합니다.\n주 증상에 대해 Thompson 씨는 일상 활동에 지장을 줄 정도로 지속적이고 심한 피로감을 느끼며, 그 정도는 10점 만점에 8점이라고 표현했습니다. 또한 주로 무릎과 손목에 관절통이 있는데, 아침에 더 심하고 하루 중 활동하면서 점차 나아진다고 합니다. 관절 부위의 붓기나 붉어짐은 없다고 합니다.\n임상 소견을 바탕으로 Thompson 씨의 피로감과 관절통의 원인을 찾기 위한 추가 검사가 필요합니다. 혈액 검사, 영상 검사 및 필요시 전문의 상담을 진행할 예정입니다.\"\n\n담당의: ABC 의사\n날짜: 2099년 7월 10일\n환자 ID: 246813579\n주 증상: 임신 정기 검진\n현재 질환 이력:\n환자 Emily Adams 씨(30세, 여성)는 정기 임신 검진을 위해 내원하였습니다. 현재 임신 32주차로, 2099년 8월 27일이 출산 예정일입니다. 기혼자로 남편과 함께 살고 있습니다.\n진료 중 Adams 씨는 

{'input_documents': [Document(id='06189ba5-40aa-4bf6-81f6-d6344f261d65', metadata={'source': 'mocked_up_physician_records_ko.csv'}, page_content='담당의: ABC 의사\n날짜: 2099년 11월 15일\n환자 ID: 123456789\n주 증상: 피로감과 관절통\n현재 질환 이력:\n환자 Sarah Thompson 씨(57세, 여성)는 피로감과 관절통을 호소하며 내원하였습니다. 홀로 거주 중인 미망인으로, 최근 해외여행 이력은 없습니다.\n진료 중 Thompson 씨는 자매가 류마티스 관절염 진단을 받은 자가면역 질환 가족력이 있다고 밝혔습니다. 또한 본인은 갑상선 기능 저하증이 있어 갑상선 호르몬 대체 요법으로 관리 중이라고 합니다.\n주 증상에 대해 Thompson 씨는 일상 활동에 지장을 줄 정도로 지속적이고 심한 피로감을 느끼며, 그 정도는 10점 만점에 8점이라고 표현했습니다. 또한 주로 무릎과 손목에 관절통이 있는데, 아침에 더 심하고 하루 중 활동하면서 점차 나아진다고 합니다. 관절 부위의 붓기나 붉어짐은 없다고 합니다.\n임상 소견을 바탕으로 Thompson 씨의 피로감과 관절통의 원인을 찾기 위한 추가 검사가 필요합니다. 혈액 검사, 영상 검사 및 필요시 전문의 상담을 진행할 예정입니다."'),
  Document(id='b29ba404-e5c2-4115-befe-7790fc8eb705', metadata={'source': 'mocked_up_physician_records_ko.csv'}, page_content='담당의: ABC 의사\n날짜: 2099년 7월 10일\n환자 ID: 246813579\n주 증상: 임신 정기 검진\n현재 질환 이력:\n환자 Emily Adams 씨(30세, 여성)는 정기 임신 검진을 위해 내원하였습니다. 현재 임신 32주차로, 2099년 8월 27일이 출산 예정일입니다. 기혼자로 남편과 함께 살고 

In [44]:
docs[0]

Document(id='06189ba5-40aa-4bf6-81f6-d6344f261d65', metadata={'source': 'mocked_up_physician_records_ko.csv'}, page_content='담당의: ABC 의사\n날짜: 2099년 11월 15일\n환자 ID: 123456789\n주 증상: 피로감과 관절통\n현재 질환 이력:\n환자 Sarah Thompson 씨(57세, 여성)는 피로감과 관절통을 호소하며 내원하였습니다. 홀로 거주 중인 미망인으로, 최근 해외여행 이력은 없습니다.\n진료 중 Thompson 씨는 자매가 류마티스 관절염 진단을 받은 자가면역 질환 가족력이 있다고 밝혔습니다. 또한 본인은 갑상선 기능 저하증이 있어 갑상선 호르몬 대체 요법으로 관리 중이라고 합니다.\n주 증상에 대해 Thompson 씨는 일상 활동에 지장을 줄 정도로 지속적이고 심한 피로감을 느끼며, 그 정도는 10점 만점에 8점이라고 표현했습니다. 또한 주로 무릎과 손목에 관절통이 있는데, 아침에 더 심하고 하루 중 활동하면서 점차 나아진다고 합니다. 관절 부위의 붓기나 붉어짐은 없다고 합니다.\n임상 소견을 바탕으로 Thompson 씨의 피로감과 관절통의 원인을 찾기 위한 추가 검사가 필요합니다. 혈액 검사, 영상 검사 및 필요시 전문의 상담을 진행할 예정입니다."')

**질문 #4 수정: *확실하게* 검사실 검사를 필요로 하는 환자는 누구인가요?**

In [45]:
current_query = "확실하게 검사실 검사를 필요로 하는 환자는 누구인가요?"
print(current_query)
docs = vector_db.similarity_search(current_query)
response = chain.invoke({"input_documents": docs, "question": current_query})
response

확실하게 검사실 검사를 필요로 하는 환자는 누구인가요?
[chain/start] [chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "확실하게 검사실 검사를 필요로 하는 환자는 누구인가요?",
  "context": "담당의: ABC 의사\n날짜: 2099년 11월 15일\n환자 ID: 123456789\n주 증상: 피로감과 관절통\n현재 질환 이력:\n환자 Sarah Thompson 씨(57세, 여성)는 피로감과 관절통을 호소하며 내원하였습니다. 홀로 거주 중인 미망인으로, 최근 해외여행 이력은 없습니다.\n진료 중 Thompson 씨는 자매가 류마티스 관절염 진단을 받은 자가면역 질환 가족력이 있다고 밝혔습니다. 또한 본인은 갑상선 기능 저하증이 있어 갑상선 호르몬 대체 요법으로 관리 중이라고 합니다.\n주 증상에 대해 Thompson 씨는 일상 활동에 지장을 줄 정도로 지속적이고 심한 피로감을 느끼며, 그 정도는 10점 만점에 8점이라고 표현했습니다. 또한 주로 무릎과 손목에 관절통이 있는데, 아침에 더 심하고 하루 중 활동하면서 점차 나아진다고 합니다. 관절 부위의 붓기나 붉어짐은 없다고 합니다.\n임상 소견을 바탕으로 Thompson 씨의 피로감과 관절통의 원인을 찾기 위한 추가 검사가 필요합니다. 혈액 검사, 영상 검사 및 필요시 전문의 상담을 진행할 예정입니다.\"\n\n담당의: ABC 의사\n날짜: 2099년 7월 10일\n환자 ID: 246813579\n주 증상: 임신 정기 검진\n현재 질환 이력:\n환자 Emily Adams 씨(30세, 여성)는 정기 임신 검진을 위해 내원하였습니다. 현재 임신 32주차로, 2099년 8월 27일이 출산 예정일입니다. 기혼자로 남편과 함께 살고 있습니다.

{'input_documents': [Document(id='06189ba5-40aa-4bf6-81f6-d6344f261d65', metadata={'source': 'mocked_up_physician_records_ko.csv'}, page_content='담당의: ABC 의사\n날짜: 2099년 11월 15일\n환자 ID: 123456789\n주 증상: 피로감과 관절통\n현재 질환 이력:\n환자 Sarah Thompson 씨(57세, 여성)는 피로감과 관절통을 호소하며 내원하였습니다. 홀로 거주 중인 미망인으로, 최근 해외여행 이력은 없습니다.\n진료 중 Thompson 씨는 자매가 류마티스 관절염 진단을 받은 자가면역 질환 가족력이 있다고 밝혔습니다. 또한 본인은 갑상선 기능 저하증이 있어 갑상선 호르몬 대체 요법으로 관리 중이라고 합니다.\n주 증상에 대해 Thompson 씨는 일상 활동에 지장을 줄 정도로 지속적이고 심한 피로감을 느끼며, 그 정도는 10점 만점에 8점이라고 표현했습니다. 또한 주로 무릎과 손목에 관절통이 있는데, 아침에 더 심하고 하루 중 활동하면서 점차 나아진다고 합니다. 관절 부위의 붓기나 붉어짐은 없다고 합니다.\n임상 소견을 바탕으로 Thompson 씨의 피로감과 관절통의 원인을 찾기 위한 추가 검사가 필요합니다. 혈액 검사, 영상 검사 및 필요시 전문의 상담을 진행할 예정입니다."'),
  Document(id='b29ba404-e5c2-4115-befe-7790fc8eb705', metadata={'source': 'mocked_up_physician_records_ko.csv'}, page_content='담당의: ABC 의사\n날짜: 2099년 7월 10일\n환자 ID: 246813579\n주 증상: 임신 정기 검진\n현재 질환 이력:\n환자 Emily Adams 씨(30세, 여성)는 정기 임신 검진을 위해 내원하였습니다. 현재 임신 32주차로, 2099년 8월 27일이 출산 예정일입니다. 기혼자로 남편과 함께 살고 